In [1]:
import pickle, json
import pandas as pd
import numpy as np
import random

In [12]:
num_data = 10
global_array = []
for i in range(num_data):
    fname = f"video{str(i)}.mp4"
    coordinates = []
    total_frames = 50
    for frame_idx in range(total_frames):
        coordinates.append(np.random.rand(66).tolist())
    
    label = random.choice([i for i in range(10)])
    cur_data = {
        "fname" : fname,
        "coordinates" : coordinates,
        "label" : label
    }
    global_array.append(cur_data)


    



In [13]:
# write data in a pickle file
with open("example_data.pkl", "wb") as f:
    pickle.dump(global_array, f)
    
# write data in a json file
with open("example_data.json", "w") as f:
    json.dump(global_array, f, indent=4)

In [15]:
np.array(global_array[0]["coordinates"]).shape

(50, 66)

In [2]:
from dataset import DanceDataset

In [3]:
dancedataset = DanceDataset("example_data.pkl")

In [4]:
dancedataset[0]

{'fname': 'video0.mp4',
 'coordinates': tensor([[0.7114, 0.4358, 0.4499,  ..., 0.6266, 0.5199, 0.9628],
         [0.4437, 0.5269, 0.8189,  ..., 0.7297, 0.2234, 0.9955],
         [0.6388, 0.5238, 0.0578,  ..., 0.3772, 0.4718, 0.2754],
         ...,
         [0.9402, 0.7630, 0.1226,  ..., 0.7778, 0.4855, 0.9214],
         [0.3299, 0.5576, 0.9368,  ..., 0.9400, 0.4183, 0.2952],
         [0.7541, 0.5412, 0.2441,  ..., 0.1254, 0.1625, 0.2213]]),
 'label': 1}

In [26]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class LSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim1, hidden_dim2, output_dim, n_layers,
                 bidirectional, dropout):
        super().__init__()
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim1,
                            num_layers=n_layers,
                            bidirectional=bidirectional,
                            batch_first=True)
        self.fc1 = nn.Linear(hidden_dim1 * 2, hidden_dim2)
        self.fc2 = nn.Linear(hidden_dim2, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # packed_embedded = pack_padded_sequence(text, text_lengths, batch_first=True) 

        packed_output, (hidden, cell) = self.lstm(x)
        cat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        rel = self.relu(cat)
        dense1 = self.fc1(rel)
        drop = self.dropout(dense1)
        preds = self.fc2(drop)
        return preds

In [27]:
lstm = LSTM(66, 512, 256, 10, 1,
                 True, 0.1)

In [29]:
import torch
lstm(torch.stack([dancedataset[0]["coordinates"]]))

tensor([[ 0.0457, -0.0579, -0.0036,  0.0140, -0.0230, -0.0622,  0.0122, -0.0020,
          0.0284,  0.0002]], grad_fn=<AddmmBackward0>)

In [30]:
from dataloader import DanceDataLoader

ModuleNotFoundError: No module named 'torch.util'

In [32]:
from torch.data import DataLoader

ModuleNotFoundError: No module named 'torch.data'

In [33]:
import torch

In [34]:
!pip install torch --upgrade

^C
